In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/MyDrive/Capstone

/content/gdrive/MyDrive/Capstone


# Loading the dataset

In [26]:
import pandas
import numpy as np

df = pandas.read_csv('data.csv', index_col='NUM')

In [27]:
del df["NAME"]
del df["COUNTRY"]

In [28]:
df

,Mine Annual Production (Million Tonne),Stripping Ratio,Mill Annual Production (Thousand Tonne),Reserve Mean Grade % Cu EQU.,LOM,CAPEX US$ millions
NUM,,,,,,
1,3,3.80,233,2.84,18,442
2,20,3.50,238,0.56,20,509
3,25,0.24,372,0.42,50,1983
4,22,3.00,1103,1.71,30,5628
5,33,1.31,481,0.43,31,1826
6,73,0.75,1250,0.61,33,5913
7,14,2.12,230,0.50,13,941
8,30,0.70,400,0.49,20,1606
9,25,3.00,275,0.36,20,1806


In [29]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score

data, test = train_test_split(
    df,
    test_size = 0.2,
    random_state = 5)

print(data.shape)
print(test.shape)

(41, 6)
(11, 6)


In [38]:
X = data.loc[:,['Mine Annual Production (Million Tonne)',
       'Stripping Ratio', 'Mill Annual Production (Thousand Tonne)',
       'Reserve Mean Grade % Cu EQU.', 'LOM']]
y = data["CAPEX US$ millions"]

In [39]:
df.describe()

,Mine Annual Production (Million Tonne),Stripping Ratio,Mill Annual Production (Thousand Tonne),Reserve Mean Grade % Cu EQU.,LOM,CAPEX US$ millions
count,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000
mean,32.826923,1.746923,484.019231,0.572885,25.884615,2483.711538
std,15.535817,1.103075,289.043350,0.399143,8.304626,1550.969012
min,3.000000,0.200000,185.000000,0.210000,13.000000,414.000000
25%,22.000000,0.947500,289.500000,0.412500,20.000000,1596.000000
50%,31.500000,1.440000,434.000000,0.500000,22.500000,1986.000000
75%,39.250000,2.350000,557.250000,0.585000,30.000000,3068.250000
max,74.000000,4.200000,1731.000000,2.840000,50.000000,6178.000000


In [31]:
num_folds = 6

kf = KFold(n_splits=num_folds, shuffle=True, random_state=5)

for c in kf.split(data):
    print(c)

(array([ 0,  1,  2,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       21, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]), array([ 3,  4,  6, 20, 22, 23, 24]))
(array([ 1,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39]), array([ 0,  2,  5, 26, 28, 34, 40]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17,
       20, 21, 22, 23, 24, 26, 27, 28, 30, 31, 32, 33, 34, 35, 37, 38, 40]), array([10, 18, 19, 25, 29, 36, 39]))
(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 14, 15, 16, 17, 18, 19,
       20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 34, 35, 36, 37, 38, 39, 40]), array([ 1, 11, 13, 21, 31, 32, 33]))
(array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 13, 14, 16, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 28, 29, 31, 32, 33, 34, 35, 36, 38, 39, 40]), array([ 7, 12, 15, 17, 27, 30, 37]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7, 10, 11, 12

# K-fold Training, Validation, Optimisation

In [ ]:
# Metrics:
#   RMSE -> Root Mean Squared Error
#   R2 -> Coefficient of Correlation
#   MAE -> Mean Absolute Error
#   APE -> Absolute Error

In [6]:
from sklearn.ensemble import RandomForestRegressor

In [9]:
rf = RandomForestRegressor()
scores = cross_val_score(rf, X, y, cv=kf)
print(scores)
print(np.mean(scores))


# average the scores for the model
# then use scores for the optimisation algorithms

[ 0.65403753  0.35460739 -1.74235132  0.08787399  0.45703636  0.54369026]
0.05914903891246356


In [ ]:
# ANN

In [40]:
# SVM
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

sv_regressor = SVR()
score_1 = cross_val_score(sv_regressor, X, y, cv= kf)
print(score_1)
print(np.mean(score_1))
# turn out really bad => required different approach

[-0.02820759 -0.38232329 -0.11914715 -0.1109103  -0.4421397  -0.0915511 ]
-0.1957131876858246


In [ ]:
# CART Tree (decision tree)

In [ ]:
# Also look at what hyperparameters we can optimise for each.

# Testing

In [ ]:
#use optimal hyperparameters to train model, then test

To add later on if time:  
-preprocessing (outliers, etc.)  
-ensemble methods (ANN + SVM)